In [3]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Kubeflow Pipelines MNIST example

Run this script to compile pipeline
"""


import kfp.dsl as dsl

TENSORBOARD_PATH = '/mnt'

@dsl.pipeline(
  name='MNIST',
  description='A pipeline to train and serve the MNIST example.'
)
def mnist_pipeline(epochs):

    train_task = dsl.ContainerOp(
        name='train',
        image='docker.io/170642/pytorch-mnist-tb',
        command=['python3', '/opt/pytorch-mnist/mnist.py'],
        arguments=[
            "--epochs", epochs,
            "--workflow_uid", "{{workflow.uid}}"
            ],
        pvolumes={"/mnt": dsl.PipelineVolume(pvc="tb-logs")},
        file_outputs={
                "runs_train": TENSORBOARD_PATH
            }
    )
    log_dir = dsl.InputArgumentPath(
                train_task.outputs['runs_train'],
                path=TENSORBOARD_PATH)
                
    visualize = dsl.ContainerOp(
        name='visualize',
        image='docker.io/170642/pytorch-mnist-tb',
        command=['python3', 'tensorboard_comp.py'],
        arguments=[
            "--logdir", log_dir,
            ],
        file_outputs={
                "mlpipeline-metrics": '/mnt/mlpipeline-ui-metadata.json'
            }
    )

if __name__ == '__main__':
  import kfp.compiler as compiler
  compiler.Compiler().compile(mnist_pipeline,  'mnist_pipeline.tar.gz')
